In [1]:
import json
import os
import pandas as pd
import random
import sys
from datetime import date
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import utils.utils as utils

In [2]:
# params
# st_mdl = "gpt-3.5-turbo-1106" # model string
st_mdl = "gpt-4-1106-preview"
st_task = "24game" # name of task

# construct filenames
st_today = date.today().strftime("%Y_%m_%d") # today's date
st_fn = f"{st_today}_{st_task}_n200_{st_mdl}" # file name
curr_dir = os.path.dirname(os.path.abspath("__file__"))

logger = utils.config_logger(curr_dir, st_fn)

In [3]:
# load api key from .env and create GPT object
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.getenv("OPENAI_API_KEY")
gpt = utils.GPT(openai_api_key, logger=logger, model=st_mdl)

# Load into a dictionary
# with open('data/24game.csv', 'r') as f:
#     data = json.load(f) # keys in each row: ['_id', 'answer', 'question', 'supporting_facts', 'context', 'type', 'level']

data = pd.read_csv('data/24game.csv').to_dict(orient='records') # load CSV into list of dicts

In [8]:
def compare_strings(str1, str2):
    """
    This function compares two strings after converting them to lowercase and stripping leading and trailing spaces.
    """
    return str1.lower().strip() == str(str2).lower().strip()

def check_answer(eqn):
    """
    Checks if 24 Game answer is correct
    """
    
    expr = eqn.split('=')
    try:
        if eval(expr[0]) == 24:
            return True
    except:
        pass

    return False

def check_bool(str1, bool2):
    """
        Checks if string str1 equal to bool bool2
    """
    last_idx_true = str1.rfind("TRUE")
    last_idx_false = str1.rfind("FALSE")

    # Check if "TRUE" is found and it is the last occurrence
    return last_idx_true != -1 and (last_idx_true > last_idx_false if last_idx_false != -1 else True)        

In [5]:
# dataset-specific adjustments (we eventually want a list of dicts of format [{"id": "xx", "question": "xxx", "context": "xxx"}, ...]

# pre-process data
for d in data:
    d['question'] = d['Puzzles']

df = pd.json_normalize(data)
df.head()

,id,Puzzles,AMT (s),Solved rate,1-sigma Mean (s),1-sigma STD (s),question
0,1,1 1 4 6,4.40,99.20%,4.67,1.48,1 1 4 6
1,2,1 1 11 11,4.41,99.60%,4.68,1.45,1 1 11 11
2,3,1 1 3 8,4.45,99.20%,4.69,1.48,1 1 3 8
3,4,1 1 1 8,4.48,98.80%,4.66,1.25,1 1 1 8
4,5,6 6 6 6,4.59,99.40%,4.82,1.49,6 6 6 6


## Generator

In [6]:
predictions = [] # list of predictions, reset after each set of runs
indices = [] # question numbers
evals = [] # whether prediction matches answer
questions = [] # list of questions


# Evaluate
num_questions = 200# number of questions to run on
indices = random.sample(range(len(data)), num_questions)
for idx in indices:
    item = data[idx]
    # if item['level'] == 'hard':
    # gold, question, context = item['answer'], item['question'], item['context']
    q = item['question']
    # question_with_context = ""
    # for i in range(len(context)):
    #     context_str = ''.join(context[i][1])
    #     question_with_context += f"Paragraph {i+1}: {context[i][0]} {context_str} "
    # question_with_context += f"Q: {question}"
    logger.info(q)
    
    msg = [{"role": "system", "content": "You are a mathematician playing the 24 Game. You are given 4 numbers and try to form 24 by using all 4 numbers and basic arithmetic operations ( + - * / ) to produce the number 24. Here is an example: ```Input: 8 4 3 5 \nAnswer: (8 * 4) - 3 - 5 = 24```. Respond only with an equation, do not include any words."}, {"role": "user", "content": q}]
    predictions.append(gpt.gen(msgs=msg).content)
    # answers.append(gold)
    # evals.append(check_answer(predictions[-1], gold))
    evals.append(check_answer(predictions[-1]))
    questions.append(q)
    print(predictions[-1])

# Print the predictions and answers
logger.info(predictions)
# logger.info(answers)
percent_correct = (sum(evals) / len(evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# create a dataframe with the data
# df = pd.DataFrame({'Index': indices, 'Question': questions, 'Prediction': predictions, 'Answer': answers, 'Eval': evals})
df = pd.DataFrame({'Index': indices, 'Question': questions, 'Prediction': predictions, 'Eval': evals})

# save the datafrme to csv
df.to_csv(f'results/{st_fn}.csv', index=False)

2024-02-13 23:06:53,109 - INFO - 3 8 8 9
2024-02-13 23:06:54,141 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 23:06:54,143 - INFO - 1 2 9 13


8 * (3 + 8 - 9) = 24


2024-02-13 23:06:55,316 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 23:06:55,318 - INFO - 5 8 11 13


(13 - 1) * (9 - 2) = 24


2024-02-13 23:06:56,361 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 23:06:56,363 - INFO - 2 7 8 11


...

12 / (1 - (9 / 10)) = 24


2024-02-13 23:10:54,131 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 23:10:54,133 - INFO - 2 4 5 9


(12 - 1) * (2 + 1) = 24


2024-02-13 23:10:55,545 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 23:10:55,547 - INFO - ['8 * (3 + 8 - 9) = 24', '(13 - 1) * (9 - 2) = 24', '(13 - 11) * 5 * 8 = 24', '(11 + 2) * (8 - 7) = 24', '(10 - 3) * (6 / 2) = 24', '(9 - (2 / 4)) * 12 = 24', '(10 - (3 - 4/4)) * 4 = 24', '(12 / (12 - 10)) * 8 = 24', '8 * (7 - (3 / 3)) = 24', '(13 - 4) * (10 - 5) = 24', '(6/1 - (6-5)) * 6 = 24', '6 / (1 - 1/2) = 24', '(6 / (6 - 4)) * 12 = 24', '(9 - (10 - 4 / 2)) * 2 = 24', '(13 / ((13 / 13) + 10)) = 24', '(5 * 2) * (4 - 1) = 24', '(4 * 8) + (4 * 8) / 8 = 24', '(8 * 10) - 13 - 6 = 24', '(13 - 5) * (4 - 3) = 24', '(6 * 6) - (10 + 6) = 24', '8 * (4 - 3) * 3 = 24', '(7 / 7) + 12 + 13 = 24', '(9 / (13 - (9 / 7))) = 24', '(4 * 12) / (13 - 3) = 24', '(3 + 1) * (3 + 3) = 24', '(1 + 3) * (9 - 7) = 24', '(6 / (1 - (13 / 6))) = 24', '(8 * (11 - 9)) * 11 = 24', '(10 - 9/9) * 6 = 24', '7 * (5 - 3) * 4 = 24', '(5 - (13 - 10)) * 2 = 24', '8 * (6 - (5 - 2))

(5 - 2) * 4 * 2 = 24
Percentage of True values: 11.0%


## Validator

In [7]:
gen_data_fn = f"results/{st_fn}.csv" # change this to the filename of the generator results
# gen_data_fn = 'results/2024_01_24_n200_gpt-3.5-turbo-1106_with_sols.csv'

df = pd.read_csv(gen_data_fn)
indices = df.iloc[:, 0].tolist()
indices = indices[:-1] # just because we manually added an additional row on the CSV when calculating scores

val_preds = [] # list of predictions for validator, reset after each set of runs
val_evals = [] # whether prediction matches answer


# Evaluate
for _, row in df.iterrows():
    idx = row['Index']
    # ans = row['Prediction']
    ans = row['Solution']
    gold = row['Eval']
    
    # question, context = item['question'], item['context']
    q = "Input: " + row['Question'] + " "
    # question_with_context = ""
    # for i in range(len(context)):
    #     context_str = ''.join(context[i][1])
    #     question_with_context += f"Paragraph {i+1}: {context[i][0]} {context_str} "
    # question_with_context += f"Q: {question}"
    # question_with_context += f"A: {ans}"
    q += f"Answer: {ans}"
    logger.info(q)
    msg = [{"role": "system", "content": "You are a mathematician playing the 24 Game. You are given 4 numbers and try to form 24 by using all 4 numbers and basic arithmetic operations ( + - * / ) to produce the number 24. Here is an example: ```Input: 8 4 3 5 \nAnswer: (8 * 4) - 3 - 5 = 24```. The following is a student's answer. Answer 'TRUE' if the answer is correct and 'FALSE' if incorrect."}, {"role": "user", "content": q}]
    val_preds.append(gpt.gen(msgs=msg).content)
    val_evals.append(check_bool(val_preds[-1], gold))
    print(val_preds[-1])

# Print the predictions and answers
logger.info(val_preds)
logger.info(df['Eval'].tolist())
percent_correct = (sum(val_evals) / len(val_evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# add validator predictions to column
df['Validator Pred'] = val_preds
df['Validator Eval'] = val_evals

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val.csv', index=False);

2024-02-13 23:49:57,695 - INFO - Input: 3 8 8 9 Answer: 3 * 8 * ( 9 - 8 ) = 24
2024-02-13 23:50:03,377 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 23:50:03,380 - INFO - Input: 1 2 9 13 Answer: 1 * 13 + 2 + 9 = 24


FALSE

The student's answer simplifies to:

3 * 8 * (9 - 8) = 3 * 8 * 1 = 24

However, this is incorrect because:

3 * 8 * 1 = 24 * 1 = 24

We can see that the middle term is not needed at all. The correct simplification is:

3 * 8 = 24

So the student's use of the numbers does not lead to a correct solution


2024-02-13 23:50:08,525 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 23:50:08,527 - INFO - Input: 5 8 11 13 Answer: ( 11 - 13 + 5 ) * 8 = 24


Let's check the student's answer step-by-step:

1 * 13 + 2 + 9 = 13 + 2 + 9
                  = 15 + 9
                  = 24

The student's answer is indeed correct as it reaches the target number of 24.

Answer: TRUE


2024-02-13 23:50:11,054 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 23:50:11,057 - INFO - Input: 2 7 8 11 Answer: 11 - 2 + 7 + 8 = 24


...

Let's check the student's answer by computing the expression:

The expression given is: ( 1 + 10 - 9 ) * 12

First step, within the parentheses: 1 + 10 - 9 = 11 - 9 = 2

Second step, multiply by 12: 2 * 12 = 24

The student's answer is indeed correct because the result is 24.

Answer: TRUE


2024-02-14 00:06:27,969 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:06:27,971 - INFO - Input: 2 4 5 9 Answer: ( 2 + 4 ) * ( 9 - 5 ) = 24


Let's check the student's answer by performing the operations in the correct order, following the mathematical order of operations (parentheses, exponents, multiplication and division from left to right, addition and subtraction from left to right):

1 - 1 + 12 * 2 = 0 + 24 = 24

The student's answer does indeed result in 24. Therefore, the answer is correct.

TRUE


2024-02-14 00:06:33,214 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:06:33,216 - INFO - ["FALSE\n\nThe student's answer simplifies to:\n\n3 * 8 * (9 - 8) = 3 * 8 * 1 = 24\n\nHowever, this is incorrect because:\n\n3 * 8 * 1 = 24 * 1 = 24\n\nWe can see that the middle term is not needed at all. The correct simplification is:\n\n3 * 8 = 24\n\nSo the student's use of the numbers does not lead to a correct solution", "Let's check the student's answer step-by-step:\n\n1 * 13 + 2 + 9 = 13 + 2 + 9\n                  = 15 + 9\n                  = 24\n\nThe student's answer is indeed correct as it reaches the target number of 24.\n\nAnswer: TRUE", "The student's answer is:\n\n(11 - 13 + 5) * 8\n\nFirst, we need to solve the expression inside the parentheses:\n\n11 - 13 + 5 = -2 + 5 = 3\n\nNow, we have:\n\n3 * 8 = 24\n\nSo the student's answer is TRUE.", "Let's evaluate the student's answer step by step:\n\n11 - 2 + 7 + 8\n\nFirst, compute

Let's evaluate the student's answer:

(2 + 4) * (9 - 5) = (6) * (4) = 24

This statement is TRUE because the solution provided correctly computes to 24 by using all 4 numbers with basic arithmetic operations.
Percentage of True values: 11.0%


In [10]:
fn_sfx = "" # temp
df = pd.read_csv(f'results/{st_fn}_with_val{fn_sfx}.csv')

pred_evals = []
for _, row in df.iterrows():
    pred_evals.append(check_bool(row['Validator Pred'], gold))

df['Validator Eval'] = pred_evals

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val{fn_sfx}.csv', index=False);